In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkConf, SparkContext
import glob2
import re, string
import random
import numpy as np

In [2]:
conf = SparkConf().setMaster("local").setAppName("LSH")
sc = SparkContext(conf = conf)

In [3]:
article_list = glob2.glob("athletics\\athletics\\*.txt")
article_list = article_list[0:50]

read the path of the articles

In [4]:
def create_shingle(article_dir):
    art = sc.textFile(article_dir).collect()
    name = article_dir.split("\\")[-1].split(".")[0]
    #print("file : ", name)
    shingle = []
    for i in range(0,len(art),2):
        #print("ith paragraph : ",i)
        out = re.sub('[%s]' % re.escape(string.punctuation), '', art[i])
        word = out.split()
        for j in range(len(word)):
            key = tuple(word[j:j+3])
            shingle.append((key,[name]))
            if (len(word)-j) == 3:
                break
    return shingle

Shingles = []
for file in article_list:
    shin = create_shingle(file)
    Shingles = Shingles + shin
random.shuffle(Shingles)

read the article and remove the punctuations and then split it into (3 shingles, file_name) and random suffle all shingles
to prevent some potential bias 

In [5]:
All_shingles = sc.parallelize(Shingles)
All_shingles = list(All_shingles.reduceByKey(lambda p,q:p+q).collectAsMap().values())

merge replicate singles and transform it into a list of to present the charateristic matrix

In [6]:
Charateristic_Matrix = []
for i in range(len(All_shingles)):
    for j in All_shingles[i]:
        x = (j,[i])
        Charateristic_Matrix.append(x)
Charateristic_Matrix = sc.parallelize(Charateristic_Matrix)
Charateristic_Matrix = Charateristic_Matrix.reduceByKey(lambda p,q:p+q)

transform the characteristic matrix into (file_names, col_index) 
where the column index represents the index where the single locates

In [7]:
def MinHashing(random_vector_ab, prime_num, num_shingles):
    
    def min_hash(x):
        key = x[0]
        Signature = []
        for j in random_vector_ab:
            min_hash = []
            for i in x[1]:
                v = ((i*j[0]+j[1])%prime_num)%num_shingles
                min_hash.append(v)
            Signature.append(min(min_hash))
        return (key,Signature)
    return min_hash

define a min hash function that runs through all hash functions and col_index and take the minimum one and form a list
to present as signature matrix

In [8]:
random_vec_ab = np.random.randint(low=0, high=12983, size=(100,2))
prime_number = 12983
num_shingles = len(All_shingles)
Signature_Matrix = Charateristic_Matrix.map(MinHashing(random_vec_ab,prime_number, num_shingles))

get the signature matrix 

In [9]:
S = Signature_Matrix.collectAsMap()

In [10]:
def bucket_hashing(x):
    num_bucket = 5000
    prime_num = 5851
    ab = np.random.randint(low=0, high=prime_num, size=(3,))
    
    for i in range(0,len(x[1]),2):
        v = ((np.dot(x[1][i:i+2],ab[0:2])+ab[-1])%prime_num)%num_bucket
        yield (v, [int(x[0])])

define a function that hash the column of each file in 50 bands and 2 rows into buckets and return (bucket_num, file_name)

In [11]:
Buckets = Signature_Matrix.flatMap(bucket_hashing).reduceByKey(lambda p,q:p+q).sortByKey().collect()

after getting the buckets and the file_names inside of it we merge ones with same bucket

In [14]:
Candidates = []
for i in Buckets:
    if len(i[1]) > 1:
        Candidates.append(i)

find out the candidate pairs that have been hash to the same bucket

In [15]:
def Jaccard_similarity(list_1, list_2):
    intersection = len(list(set(list_1).intersection(list_2)))
    union = len(list_1) + len(list_2) - intersection
    return float(intersection/union)

define a function thay calculate the Jaccard Similarity

In [16]:
def Simiarities(candidate, Signature):
    All_pairs = {}
    for cand in candidate:
        for i in range(len(cand[1])):
            for j in range(i, len(cand[1])):
                if i == j or cand[1][i] == cand[1][j]:
                    continue
                sim = Jaccard_similarity(Signature[str(cand[1][i]).zfill(3)],Signature[str(cand[1][j]).zfill(3)])
                All_pairs[(str(cand[1][i]).zfill(3),str(cand[1][j]).zfill(3))] = sim
    return All_pairs
All_candidate_sim = Simiarities(Candidates,S)
Sorted_Sim = {k:v for k,v in sorted(All_candidate_sim.items(), key=lambda item:item[1])}

find out the Jaccard similarity of each candidate pairs and sort by increasing orders

In [17]:
TOP_10 = list(Sorted_Sim.items())[-10:]
print("TOP 10 similar articles")
for i in range(len(TOP_10)):
    print(TOP_10[-i])

TOP 10 similar articles
(('036', '018'), 0.22699386503067484)
(('029', '002'), 0.2578616352201258)
(('036', '026'), 0.25)
(('024', '031'), 0.2422360248447205)
(('018', '031'), 0.2422360248447205)
(('036', '050'), 0.2422360248447205)
(('022', '036'), 0.2422360248447205)
(('036', '011'), 0.2345679012345679)
(('039', '002'), 0.2345679012345679)
(('022', '029'), 0.2345679012345679)


In [18]:
sc.stop()